In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [2]:
path_to_zips = '/home/dsc/Data/challenger'

In [6]:
! ls {path_to_zips}

bookings.csv.bz2  searches.csv.bz2


## Ejercicio 1

In [9]:
import bz2

newfile = bz2.BZ2File(path_to_zips + "/bookings.csv.bz2")

n = 0
for l in newfile:
    n = n + 1

In [10]:
n

10000011

## Ejercicio 2

In [11]:
Fichero de bookings con columna 
arr_port --> Aeropuerto
pax --> Pasajeros

In [31]:
df = pd.read_csv(path_to_zips + "/bookings.csv.bz2", delimiter='^',usecols=["arr_port","pax"])


In [21]:
df.sample(5)

,arr_port,pax
15144,BCN,1
49376,LAX,3
34381,LIM,1
36815,AJA,4
39498,LBV,-1


In [32]:
df_sum = df.groupby("arr_port").sum().sort("pax")

In [37]:
df2 = df_sum.reset_index()

In [39]:
df2.sort('pax',ascending=False).head(10)

,arr_port,pax
2273,LHR,88809
2272,MCO,70930
2271,LAX,70530
2270,LAS,69630
2269,JFK,66270
2268,CDG,64490
2267,BKK,59460
2266,MIA,58150
2265,SFO,58000
2264,DXB,55590


In [ ]:
# No cabe en memoria, hacerlo por chunks

In [71]:
# Objeto para iterar y cada iteración devuelve un dataframe
df = pd.read_csv(path_to_zips + "/bookings.csv.bz2", delimiter='^',usecols=["arr_port","pax"],chunksize=100000)

In [67]:
# Primera solucion: Concatenación de los top 10
l = []
for chunk in df:
    l.append(chunk.fillna(0).groupby('arr_port').sum().reset_index().sort('pax',ascending=False).head(10))

total = pd.concat(l)
total_2 = total.groupby('arr_port').sum().reset_index()
total_2.sort('pax',ascending=False).head(10)

In [ ]:
# Segunda solucion: Concatenación de los agrupados
l = []
for chunk in df:
    l.append(chunk.fillna(0).groupby('arr_port').sum().reset_index()

total = pd.concat(l)
total_2 = total.groupby('arr_port').sum().reset_index()
total_2.sort('pax',ascending=False).head(10)

In [72]:
# Tercera solucion: Concatenación de los agrupados, pero con tamaño constante
primera_iteracion = True

for chunk in df:
    s = chunk.fillna(0).groupby('arr_port').sum().reset_index()
    
    if primera_iteracion == True:
        total = s
        primera_iteracion = False
    else:
        total = pd.concat([total,s])
        
    total = total.groupby('arr_port').sum().reset_index()

total_2 = total.groupby('arr_port').sum().reset_index()
total_2.sort('pax',ascending=False).head(10)

,arr_port,pax
1088,LHR,88809
1190,MCO,70930
1050,LAX,70530
1047,LAS,69630
886,JFK,66270
315,CDG,64490
216,BKK,59460
1228,MIA,58150
1719,SFO,58000
517,DXB,55590


In [110]:
top = total_2.sort('pax',ascending=False).head(10)

In [40]:
## Ejercicio 3: Uso de Geobase

In [ ]:
# Instalación de geobase: sudo pip install geobases
# En línea de comandos geobase + aeropuerto y se obtiene información del aeropuerto

In [74]:
!GeoBase MAD

Loading "ori_por"...
Looking for matches from MAD...
Keeping 1 result(s) from 1 initially...

__ref__                                 0                                       
__key__                                 MAD                                     
__dup__                                 ['MAD@1']                               
__par__                                 []                                      
__lno__                                 7379                                    
iata_code                               MAD                                     
icao_code                               LEMD                                    
faa_code                                                                        
is_geonames                             Y                                       
geoname_id                              6299345                                 
valid_id                                                                        
name           

In [75]:
from GeoBases import GeoBase

In [76]:
g = GeoBase(data='ori_por',verbose=False)

In [119]:
g.get('MAD')['city_name_ascii']
g.get('MAD')['name']

'Madrid / Barajas'

In [113]:
def nombre_aeropuerto(codigo):
    return g.get(codigo.strip())['name']

In [115]:
top['names_map'] = top['arr_port'].map(nombre_aeropuerto)
top

,arr_port,pax,arr_port_good,names,cities,countries,names_map
1088,LHR,88809,NaN,London Heathrow Airport,London,United Kingdom,London Heathrow Airport
1190,MCO,70930,NaN,Orlando International Airport,Orlando,United States,Orlando International Airport
1050,LAX,70530,NaN,Los Angeles International Airport,Los Angeles,United States,Los Angeles International Airport
1047,LAS,69630,NaN,Mc Carran International Airport,Las Vegas,United States,Mc Carran International Airport
886,JFK,66270,NaN,John F Kennedy International Airport,New York City,United States,John F Kennedy International Airport
315,CDG,64490,NaN,Paris - Charles-de-Gaulle,Paris,France,Paris - Charles-de-Gaulle
216,BKK,59460,NaN,Suvarnabhumi,Bangkok,Thailand,Suvarnabhumi
1228,MIA,58150,NaN,Miami International Airport,Miami,United States,Miami International Airport
1719,SFO,58000,NaN,San Francisco International Airport,San Francisco,United States,San Francisco International Airport
517,DXB,55590,NaN,Dubai International Airport,Dubai,United Arab Emirates,Dubai International Airport


In [112]:
cities = top.arr_port.apply(lambda a: g.get(a.strip())['city_name_ascii'])
countries = top.arr_port.map(lambda a: g.get(a.strip())['country_name'])

top['cities'] = cities
top['countries'] = countries